In [2]:
# REF: https://machinelearningmastery.com/building-a-plain-seq2seq-model-for-language-translation/

In [18]:
import os
import requests
import unicodedata 
import zipfile 
import os
import tokenizers
import numpy as np
import random

In [5]:
#import sys
#!{sys.executable} -m pip install tokenizers

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 17.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.9 MB/s  0:00:00m 12.3 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [tokenizers] 3/5 [huggingface-hub]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [7]:
def normalize(line):
    line = unicodedata.normalize("NFKC", line.strip().lower())
    eng, spa = line.split("\t")[:2]
    return eng.lower().strip(), spa.lower().strip()

text_pairs = []
with zipfile.ZipFile("spa-eng.zip", "r") as zip_ref:
    for k, line in enumerate(zip_ref.read("spa.txt").decode("utf-8").splitlines(),start=1):
        print(line, end="\r")
        eng, spa = normalize(line)
        text_pairs.append((eng, spa))
    print(k)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
en_tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
es_tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
# config tokenizer
en_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space=True)
es_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space=True)

# config decoder
en_tokenizer.decoder = tokenizers.decoders.ByteLevel()
es_tokenizer.decoder = tokenizers.decoders.ByteLevel()

# train BPE foen eng spa 
VOCAB_SIZE = 8000
trainer = tokenizers.trainers.BpeTrainer(vocab_size=VOCAB_SIZE,  special_tokens=["[start]", "[end]", "[pad]"],
                                        show_progress=True)
en_tokenizer.train_from_iterator([x[0] for x in text_pairs], trainer=trainer)
es_tokenizer.train_from_iterator([x[1] for x in text_pairs], trainer=trainer)

en_tokenizer.enable_padding(pad_id=en_tokenizer.token_to_id("[pad]"), pad_token="[pad]")
es_tokenizer.enable_padding(pad_id=es_tokenizer.token_to_id("[pad]"), pad_token="[pad]")
# save trained tokenizers
en_tokenizer.save("en_tokenizer.json", pretty=True)
es_tokenizer.save("es_tokenizer.json", pretty=True)


In [20]:
print("Sample Tokenization:")
en_sample, es_sample = random.choice(text_pairs)
encoded = en_tokenizer.encode(en_sample)

print(f"Original: {en_sample}")
print(f"Tokens: {encoded.tokens}")
print(f"IDs: {encoded.ids}")
print(f"Decoded: {en_tokenizer.decode(encoded.ids)}")
print()
 
encoded = es_tokenizer.encode("[start] " + es_sample + " [end]")
print(f"Original: {es_sample}")
print(f"Tokens: {encoded.tokens}")
print(f"IDs: {encoded.ids}")
print(f"Decoded: {es_tokenizer.decode(encoded.ids)}")
print()

Sample Tokenization:
Original: tom didn't know anything about what was happening.
Tokens: ['Ġtom', 'Ġdidn', "'t", 'Ġknow', 'Ġanything', 'Ġabout', 'Ġwhat', 'Ġwas', 'Ġhappening', '.']
IDs: [96, 261, 111, 192, 457, 269, 160, 142, 2023, 14]
Decoded:  tom didn't know anything about what was happening.

Original: tom no sabía nada acerca de lo que estaba pasando.
Tokens: ['[start]', 'Ġtom', 'Ġno', 'ĠsabÃŃa', 'Ġnada', 'Ġacerca', 'Ġde', 'Ġlo', 'Ġque', 'Ġestaba', 'Ġpasando', '.', 'Ġ', '[end]']
IDs: [0, 114, 113, 706, 367, 1109, 97, 153, 111, 285, 2104, 14, 72, 1]
Decoded:  tom no sabía nada acerca de lo que estaba pasando. 



In [17]:
text_pairs

[('go.', 've.'),
 ('go.', 'vete.'),
 ('go.', 'vaya.'),
 ('go.', 'váyase.'),
 ('hi.', 'hola.'),
 ('run!', '¡corre!'),
 ('run!', '¡corran!'),
 ('run!', '¡huye!'),
 ('run!', '¡corra!'),
 ('run!', '¡corred!'),
 ('run.', 'corra.'),
 ('run.', 'corred.'),
 ('who?', '¿quién?'),
 ('wow!', '¡órale!'),
 ('duck!', '¡inclínate!'),
 ('duck!', '¡agáchate!'),
 ('duck!', '¡agáchese!'),
 ('duck!', '¡agáchense!'),
 ('duck!', '¡agachaos!'),
 ('fire!', '¡fuego!'),
 ('fire!', '¡incendio!'),
 ('fire!', '¡disparad!'),
 ('help!', '¡ayuda!'),
 ('help!', '¡socorro! ¡auxilio!'),
 ('help!', '¡auxilio!'),
 ('hide.', 'escóndete.'),
 ('jump!', '¡salta!'),
 ('jump.', 'salte.'),
 ('jump.', 'salto.'),
 ('stay.', 'quédate.'),
 ('stay.', 'quédese.'),
 ('stay.', 'quédense.'),
 ('stay.', 'quedaos.'),
 ('stop!', '¡parad!'),
 ('stop!', '¡para!'),
 ('stop!', '¡pare!'),
 ('wait!', '¡espera!'),
 ('wait!', '¡esperen!'),
 ('wait!', '¡espérate!'),
 ('wait.', 'esperen.'),
 ('wait.', 'espera.'),
 ('wait.', 'espere.'),
 ('wait.', 'esp

In [ ]:
np.random.ch